# Financial Fraud Detection
Brev Launchable with SHAPLEY Values

- The objective of this notebook is to showcase the usage of the ___financial-fraud-training___ NIM (microservice) (NEED LINK) and how to deploy the produced trained models on the Triton Inference Server.
- We use [IBM TabFromer](https://github.com/IBM/TabFormer) as an example dataset
- That datset is then preprocess before running through the training NIM.

NOTICE:
- This notebook assume that you have followed the pre

NOTE: The preprocessing code is written specifically for the TabFormer dataset and will not work with other datasets.

In [ ]:
!pip install -r "./requirements.txt"

#### Import libraries

In [ ]:
import os
import sys
import json
import time
import subprocess

----
# Step 1: Get and Prepare the data

___This example uses the IBM TabFormer dataset.
Unfortunatley it is not a simple process to download the dataset. 
There are a few manual steps needed to access the data___

## How to get a link to the Test Data

The origin of the test data this [TabFormer git repository](https://github.com/IBM/TabFormer/tree/main/data/credit_card). The problem is that it is generally not downloadable from this location as it is a large file and is stored / retrieved using git-lfs. Due to the popularity of the file it is generally not able to be downloaded here. As such, the owners of that repository have made the dataset available via [Box](https://ibm.box.com/v/tabformer-data).

While this makes it fairly easy to retrieve the file for local use, it does make it a little more difficult to retrieve for the sakes of testing in a container as this application expects. So, there are a few steps to follow to get a download link that you can put into the download box.

## Step 1

Open a browser window and navigate to the box dataset [https://ibm.box.com/v/tabformer-data](https://ibm.box.com/v/tabformer-data). 

![Image 1](../docs/images/1.png)

Right click in the webpage and select `Inspect`. When the developer tools opens, select the `Network`

![Image 2](../docs/images/2.png)


## Step 2

In the webpage itself, click on the credit_card folder. 

![Image 3](../docs/images/3.png)

You will see `transactions.tgz` when you point at the file a 3-button menu will show up and when you click that 3-button menu, a download dialog will pop up. 

Click download and after a moment the file should start downloading locally (this is fine, you can delete it after). 

In the network panel an entry will show up by the name `download`. Click the name `download` and the `Headers` panel will show up. In there you will find `Request URL`. Highlight the entire value (**this is the download URL**) and `copy` the URL. This is a temporary link that is provided. It has a time limit. It will look something like this: `https://public.boxcloud.com/d/1/{LOTS OF RANDOM LOOKING CHARACTERS}/download`

![Image 4](../docs/images/4.png)

## Step 3

In the notebook set the value of `DOWNLOAD_URL` by pasting in the url you copied in step 2: `DOWNLOAD_URL="[the pasted url text goes in between these quotes, do not include these brackets]"`



## Download the Dataset

In [ ]:
DOWNLOAD_URL = ""

In [ ]:
# make sure we are in the "data" folder
%cd ../data
%pwd

In [ ]:
!wget {DOWNLOAD_URL}

In [ ]:
!mv download download.tgz

In [ ]:
!tar xvzf download.tgz

In [ ]:
!mv card_transaction.v1.csv ./TabFormer/raw

You should now have
```
.
    data
    └── TabFormer
        └── raw
            └── card_transaction.v1.csv
```

In [ ]:
# Once the raw data is placed as described above, set the path to the TabFormer directory

# Change this path to point to TabFormer data
data_root_dir = os.path.abspath('../data/TabFormer/') 

# Change this path to the directory where you want to save your model
model_output_dir = os.path.join(data_root_dir, 'trained_models')

# Path to save the trained model
os.makedirs(model_output_dir, exist_ok=True)

In [ ]:
# Check if the raw data has been placed properly
!tree {data_root_dir}

---
# Step 2: Preprocess the data and 
- Import the Python function that handles preprocess the TabFormer data
- Call `preprocess_TabFormer` function to prepare the data


In [ ]:
# Add the "src" directory to the search path
src_dir = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), 'src'))
sys.path.insert(0, src_dir)

# should be able to import from "src" folder now
from preprocess_TabFormer import proprocess_data

In [ ]:
# Preprocess the data
mask_mapping, feature_mask = preprocess_data(data_root_dir)

# this will output status as it correlates data and produces catagorical types
# the mask_mapping anf feature_mask variable will be used later 

In [ ]:
# You should not see files under a "gnn" folder and under a "xgb" folder
!tree {data_root_dir}

-----
# Step 3:  Use the financial-fraud-training NIM to train an XGBoost model


### Create training configuration file
NOTE: Training configuration file must conform to the training schemas defined in financial-fraud-training NIM  (NOTE:  NEED A LINK TO THE DOCS)

__Important: Models and configuration files needed for deployment using the Triton Inference server will be saved in trained_models/model-repository__

In [ ]:
training_config = {
  "paths": {
    "data_dir": "/data", # Mount dataset root directory under /data in the container
    "output_dir": "/trained_models" # Mount path to save the trained models.
                                    # NOTE: This path is inside the docker container 
  },

  "models": [
    {
      "kind": "GraphSAGE_XGBoost",
      "gpu": "single",
      "hyperparameters": {
        "gnn":{
          "hidden_channels": 16,
          "n_hops": 1,
          "dropout_prob": 0.1,
          "batch_size": 1024,
          "fan_out": 16,
          "num_epochs": 16
        },
        "xgb": {
          "max_depth": 6,
          "learning_rate": 0.2,
          "num_parallel_tree": 3,
          "num_boost_round": 512,
          "gamma": 0.0
        }

      }
    }
  ]
}


#### Save the training configuration file as a json file

In [ ]:
training_config_file_name = 'training_config.json'

with open(os.path.join(training_config_file_name), 'w') as json_file:
    json.dump(training_config, json_file, indent=4)

### Pull and run the financial-fraud-training NIM 


In [ ]:
API_KEY="replace with your API key"

In [ ]:
!docker login nvcr.io --username '$oauthtoken' --password {API_KEY}

In [ ]:
!docker pull nvcr.io/nvstaging/nim/financial-fraud-training:1.0.0-rc1

#### Create a local cache directory

In [ ]:
username = subprocess.run(["whoami"], capture_output=True, text=True).stdout.strip()
nim_cache_dir = f'/home/{username}/.cache/nim'

In [ ]:
!mkdir -p {nim_cache_dir}

#### Set container name and ports for running the container

In [ ]:
NIM_HTTP_PORT = 8002
NIM_GRPC_PORT = 50051
CONTAINER_NAME = "financial-fraud-training"

In [ ]:
# Stop any running container with the same name
!docker stop {CONTAINER_NAME}
!docker rm {CONTAINER_NAME}

#### Run the container

In [ ]:
!docker run -d -it --rm --runtime=nvidia --name={CONTAINER_NAME} --gpus all \
    -p {NIM_HTTP_PORT}:{NIM_HTTP_PORT} -e NIM_HTTP_API_PORT={NIM_HTTP_PORT} -p {NIM_GRPC_PORT}:{NIM_GRPC_PORT} \
    -e NIM_DISABLE_MODEL_DOWNLOAD=True -e NIM_GRPC_API_PORT={NIM_GRPC_PORT} -e NIM_CACHE_PATH=/opt/nim/.cache \
    -e NIM_CACHE_PATH=/opt/nim/.cache  --mount=type=bind,src={nim_cache_dir},dst=/opt/nim/.cache -v {data_root_dir}:/data \
    -v {model_output_dir}:/trained_models nvcr.io/nvstaging/nim/financial-fraud-training:1.0.0-rc1 -e NGC_API_KEY={API_KEY}

In [ ]:
time.sleep(5)

### Finally, initiate model training using the training configuration defined earlier

* Initiate training via the /train endpoint by sending the training configuration as a JSON payload.



In [ ]:
cmd = [
    "curl",
    "-X", "POST",
    "-H", "Content-Type: application/json",
    "-d", json.dumps(training_config),
    f"http://0.0.0.0:{NIM_HTTP_PORT}/train"
]
result = subprocess.run(cmd, capture_output=True, text=True)
result.stdout

In [ ]:
#!curl -X POST "http://0.0.0.0:$NIM_HTTP_PORT/train"   -H "Content-Type: application/json"   -d @{training_config_file_name}

#### Make sure that the `model_repository` has been created with right contents in it
According the above defined configuration file, the `model_repository`, which is folder containing the models and configuration files to be deployed on the Triton inference Server, will be created under 
{data_root_dir}/trained_models/ and its contents will look like

```sh
├── model
│   ├── 1
│   │   └── graph_sage_node_embedder.onnx
│   └── config.pbtxt
└── xgboost
    ├── 1
    │   └── xgboost_on_embeddings.json
    └── config.pbtxt

```


In [ ]:
!tree {model_output_dir}/model_repository

----
# Step 4:  Serve your python backend model on Triton Inference Server

!Important: Change MODEL_REPO_PATH to point to the `model python_backend_model_repository` folder if you used different path in your training configuration file

### Deploy python backend model¶


In [ ]:
if run_locally:
    HOST = 'localhost'
else:
    HOST = '<SERVER_URL>' # Replace with your server URL or IP address

HTTP_PORT = 8005
GRPC_PORT = 8006
METRICS_PORT = 8007

In [ ]:
if run_locally:
    
    # Triton server image
    TRITON_IMAGE = 'nvcr.io/nvidia/tritonserver:25.01-py3'
    MODEL_REPO_PATH = os.path.join(model_output_dir, 'python_backend_model_repository')

    # Pull docker 
    !docker pull {TRITON_IMAGE}
    !docker stop tritonserver
    !docker rm tritonserver

    !docker run --gpus all -d -p {HTTP_PORT}:{HTTP_PORT} -p {GRPC_PORT}:{GRPC_PORT} \
        -v {MODEL_REPO_PATH}:/models --name tritonserver {TRITON_IMAGE} tritonserver \
        --model-repository=/models   --http-port={HTTP_PORT} --grpc-port={GRPC_PORT} \
        --metrics-port={METRICS_PORT}

In [ ]:
client_grpc = triton_grpc.InferenceServerClient(url=f'{HOST}:{GRPC_PORT}')

In [ ]:
import subprocess

container_name = "tritonserver"

while True:
    try:
        if client_grpc.is_server_ready():
            break
    except triton_utils.InferenceServerException as e:
        pass
    try:
        # Run the docker logs command with the --tail option
        output = subprocess.check_output(["docker", "logs", "--tail", "10", container_name])
        print(output.decode("utf-8"))
    except subprocess.CalledProcessError as e:
        print("Error retrieving logs:", e)
    time.sleep(10)

## Prediction without computing Shapley values¶


In [ ]:
import os
import pandas as pd
import numpy as np

model_name = "prediction_and_shapley"
test_path = os.path.join(data_root_dir, "xgb/test.csv") # already preprocessed data
test_df = pd.read_csv(test_path)
X = test_df.iloc[:, :-1].values.astype(np.float32)
y = test_df.iloc[:, -1].values
edge_index = np.array([[], []]).astype(np.int64) # empty edge_index
compute_shap = np.array([False], dtype=bool) # Skip shap value computation

In [ ]:
feature_mask = feature_mask.astype(np.int32)

In [ ]:
with httpclient.InferenceServerClient(f"localhost:{HTTP_PORT}") as client:
    input_features = httpclient.InferInput("NODE_FEATURES", X.shape, datatype="FP32")
    input_features.set_data_from_numpy(X)

    input_edge_indices = httpclient.InferInput("EDGE_INDEX", edge_index.shape, datatype="INT64")
    input_edge_indices.set_data_from_numpy(edge_index)

    input_feature_mask = httpclient.InferInput("FEATURE_MASK", feature_mask.shape, datatype="INT32")
    input_feature_mask.set_data_from_numpy(feature_mask)

    compute_shap_flag = httpclient.InferInput("COMPUTE_SHAP", compute_shap.shape, datatype="BOOL")
    compute_shap_flag.set_data_from_numpy(compute_shap)
    
    outputs = [
        httpclient.InferRequestedOutput("PREDICTION"),
        httpclient.InferRequestedOutput("SHAP_VALUES")
    ]
    response = client.infer(model_name, inputs=[input_features, input_edge_indices, compute_shap_flag, input_feature_mask ], request_id=str(1), outputs=outputs)


In [ ]:
predictions= response.as_numpy('PREDICTION')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

y_pred = (predictions > 0.5).astype(int)


# Compute evaluation metrics
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred, zero_division=0)
recall = recall_score(y, y_pred, zero_division=0)
f1 = f1_score(y, y_pred, zero_division=0)

print("----Summary---")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [ ]:
import pandas as pd
# Create a DataFrame with labeled rows and columns
classes = ['Non-Fraud', 'Fraud']
columns = pd.MultiIndex.from_product([["Predicted"], classes])
index = pd.MultiIndex.from_product([["Actual"], classes])

conf_mat = confusion_matrix(y, y_pred)
cm_df = pd.DataFrame(conf_mat, index=index, columns=columns)
print(cm_df)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# Plot the confusion matrix directly from predictions
disp = ConfusionMatrixDisplay.from_predictions(
    y, y_pred, display_labels=classes)
disp.ax_.set_title('Confusion Matrix')
plt.show()

## Compute Shapley value for different features for a transaction

In [ ]:

# Set COMPUTE_SHAP flag to True
compute_shap = np.array([True], dtype=bool)

X = test_df.iloc[:1, :-1].values.astype(np.float32)
y = test_df.iloc[:1, -1].values

with httpclient.InferenceServerClient(f"localhost:{HTTP_PORT}") as client:
    input_features = httpclient.InferInput("NODE_FEATURES", X.shape, datatype="FP32")
    input_features.set_data_from_numpy(X)

    input_edge_indices = httpclient.InferInput("EDGE_INDEX", edge_index.shape, datatype="INT64")
    input_edge_indices.set_data_from_numpy(edge_index)

    input_feature_mask = httpclient.InferInput("FEATURE_MASK", feature_mask.shape, datatype="INT32")
    input_feature_mask.set_data_from_numpy(feature_mask)

    compute_shap_flag = httpclient.InferInput("COMPUTE_SHAP", compute_shap.shape, datatype="BOOL")
    compute_shap_flag.set_data_from_numpy(compute_shap)
    
    outputs = [
        httpclient.InferRequestedOutput("PREDICTION"),
        httpclient.InferRequestedOutput("SHAP_VALUES")
    ]
    response = client.infer(model_name, inputs=[input_features, input_edge_indices, compute_shap_flag, input_feature_mask ], request_id=str(1), outputs=outputs)


predictions= response.as_numpy('PREDICTION')
shap_values = response.as_numpy('SHAP_VALUES')

In [ ]:
feature_to_attribution_map = dict(zip(feature_mask, shap_values[0]))
feature_name_to_id_map = {v:k  for k,v in mask_mapping.items()}

#### Shapley values for different features

In [ ]:
{feature_name_to_id_map[k]: f"{v:.3f}" for k, v in feature_to_attribution_map.items()}

# Copyright
Copyright (c) 2025, NVIDIA CORPORATION. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.